# GPT-2 Transformer model: Case study of Review text generation

Before starting, it is interesting to check the capabilities we have in our server.

In [1]:
!nvidia-smi #This is the command to know the GPU info 

Mon Nov 21 10:40:32 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   63C    P8    12W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
#cat /proc/meminfo

We can also check the CPU processor we have in our machine

In [3]:
!lscpu

Architecture:        x86_64
CPU op-mode(s):      32-bit, 64-bit
Byte Order:          Little Endian
CPU(s):              2
On-line CPU(s) list: 0,1
Thread(s) per core:  2
Core(s) per socket:  1
Socket(s):           1
NUMA node(s):        1
Vendor ID:           GenuineIntel
CPU family:          6
Model:               63
Model name:          Intel(R) Xeon(R) CPU @ 2.30GHz
Stepping:            0
CPU MHz:             2299.998
BogoMIPS:            4599.99
Hypervisor vendor:   KVM
Virtualization type: full
L1d cache:           32K
L1i cache:           32K
L2 cache:            256K
L3 cache:            46080K
NUMA node0 CPU(s):   0,1
Flags:               fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge mca cmov pat pse36 clflush mmx fxsr sse sse2 ss ht syscall nx pdpe1gb rdtscp lm constant_tsc rep_good nopl xtopology nonstop_tsc cpuid tsc_known_freq pni pclmulqdq ssse3 fma cx16 pcid sse4_1 sse4_2 x2apic movbe popcnt aes xsave avx f16c rdrand hypervisor lahf_lm abm invpcid_single ssbd ibrs 

The first step is to download the neccessary libraries used in this notebook

# 1.Libraries

In [4]:
%tensorflow_version 2.x
!pip3 install gpt-2-simple

Colab only includes TensorFlow 2.x; %tensorflow_version has no effect.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [5]:
import gpt_2_simple as gpt2

In [6]:
from datetime import datetime
from google.colab import files
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split 


# 2.Download GPT2

In [7]:
gpt2.download_gpt2(model_name="124M")

Fetching checkpoint: 1.05Mit [00:00, 410Mit/s]                                                      
Fetching encoder.json: 1.05Mit [00:00, 4.47Mit/s]
Fetching hparams.json: 1.05Mit [00:00, 417Mit/s]                                                    
Fetching model.ckpt.data-00000-of-00001: 498Mit [00:10, 49.3Mit/s]                                  
Fetching model.ckpt.index: 1.05Mit [00:00, 617Mit/s]                                                
Fetching model.ckpt.meta: 1.05Mit [00:00, 6.52Mit/s]
Fetching vocab.bpe: 1.05Mit [00:00, 6.77Mit/s]


In [8]:
 #gpt2.mount_gdrive()

# 3.Create/upload text file

In [9]:
#Function to create the Datasets. 
#The datase will have a column "start" and "end" to separate texts.
#Validation or thruthful are the real datasets, used to train the model to generate fake reviews.


def train_valid_datasets(df,file_name_dtrain,file_name_dtruthful):

  df["fake"] = 0
  df_train, df_valid = train_test_split(df, random_state=42, train_size=0.75, stratify=df.overall.values)  #Separate  data (0.75/0.25)

  df_train['end']= '<|endoftext|>'
  df_train['start']= '<|start|>'

  df_train=df_train.reset_index()
  df_train=df_train[['start','reviewText','end']]
  file_train= df_train.to_csv(file_name_dtrain,index=False)  #To generate a txt file with the training data

  df_truthful=df_valid.reset_index()
  df_truthful=df_truthful[['reviewText','fake']]  
  #file_truthful= df_truthful.to_csv(file_name_dtruthful,index=False)
  
  return df_train,df_truthful

In [10]:
#from google.colab import drive
#drive.mount('/content/drive')

In [11]:
#This is to load the dataset
df = pd.read_csv('CellPhonesAccessories_truthful.txt')
df




,overall,verified,reviewTime,reviewerID,asin,style,reviewerName,reviewText,summary,unixReviewTime,vote,image,helpful
0,5.0,True,2011-05-25,A2YZIPWA3TSNA4,B004BBFPBS,{'Color:': ' Car Holder'},kapdoubleE,Can't say enough positive things about this it...,SUPER SUPER AWESOME,1306281600,2.0,NaN,NaN
1,4.0,True,2012-10-22,A2LEO7WT1VIMKZ,B002QHBAH2,"{'Size:': ' Argyle', 'Color:': ' Clear'}",jmonty,Even with shipping that's more than the cost o...,Great case for the price,1350864000,14.0,['https://images-na.ssl-images-amazon.com/imag...,NaN
2,4.0,True,2012-09-03,A1A8Z5YQ24WBFW,B00756KYD0,NaN,sami,The phone case fits the phone perfect. My son ...,Product is what I expected.,1346630400,5.0,NaN,NaN
3,2.0,True,2011-12-13,A2ME89MSWVG9NF,B005U5L5OK,NaN,Amazon Customer,When I tried out the Parrot I thought it was g...,Stop working properly after one day,1323734400,2.0,NaN,NaN
4,5.0,True,2012-09-05,A1SZSNPW4JNJEN,B0089I25YO,NaN,Sling,I have owned a lot of cases for my myriad phon...,Terrific case,1346803200,7.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,5.0,True,2011-03-25,A2IWQ4HNOZO7M8,B003WRB1C4,NaN,JimiB,"Great price. Good fit, especially with the iPh...",Works for me...,1301011200,3.0,NaN,NaN
1996,5.0,True,2012-04-24,A2P68VRKQMYBDE,B003FRTGEQ,NaN,aceforall,Very easy and simple to use and get going...\n...,VERY NICE!!,1335225600,6.0,NaN,NaN
1997,5.0,True,2012-08-11,A3NHUYF2BWMFEU,B003XDU8OO,NaN,Dana D,I've purchased about 20 different bt headsets ...,The best yet...,1344643200,8.0,NaN,NaN
1998,5.0,True,2012-01-26,A2HEHADY3O4AZN,B006MK0ZD4,"{'Color:': ' Black', 'Package Type:': ' Standa...",Terrigno,Recieved my Case today and I absolutely love i...,Adorible Android Case,1327536000,2.0,NaN,NaN


In [12]:
filename_dtrain='D_train.txt'
filename_dvalid='D_valid.txt'
df_train,df_truthful= train_valid_datasets(df,filename_dtrain,filename_dvalid)

In [13]:
df_train

,start,reviewText,end
0,<|start|>,First off i have got to say TPU has great Serv...,<|endoftext|>
1,<|start|>,I installed the antenna as per instructions an...,<|endoftext|>
2,<|start|>,Dennis loves this cover because it is made of ...,<|endoftext|>
3,<|start|>,The sound of this headset is very good for peo...,<|endoftext|>
4,<|start|>,I bought this screen protector for my Droid Bi...,<|endoftext|>
...,...,...,...
1495,<|start|>,I hate spending a lot of money on cases becaus...,<|endoftext|>
1496,<|start|>,this is the very best way to support any table...,<|endoftext|>
1497,<|start|>,After trying a lot of slider cases for the iPh...,<|endoftext|>
1498,<|start|>,First Off.. Let Me Say This IS A Well Construc...,<|endoftext|>


In [14]:
df_truthful

,reviewText,fake
0,I originally ordered screen protectors in Dece...,0
1,I used this item for little over a month and t...,0
2,I wanted something to protect my phone from bo...,0
3,I had a local stereo shop install this item.\n...,0
4,"I've went through a few cases from Case Crown,...",0
...,...,...
495,"So, as many Life Proof case owners, I was sear...",0
496,Pretty decent clear case for iPhone 4/4s. The ...,0
497,I just received this today and wanted to give ...,0
498,I purchased this phone for a couple of months ...,0


# 4.Finetune GPT2

In [15]:
file_name= filename_dtrain   #Training dataset

In [16]:
sess = gpt2.start_tf_sess()

In [17]:
gpt2.finetune(sess,
              dataset=file_name,
              model_name='124M',
              steps=1000,#1000
              restore_from='fresh',
              run_name='run1',
              print_every=10,
              sample_every=200,#200
              save_every=500)#500

Loading checkpoint models/124M/model.ckpt
Loading dataset...


100%|██████████| 1/1 [00:01<00:00,  1.59s/it]


dataset has 326809 tokens
Training...
[10 | 26.41] loss=3.17 avg=3.17
[20 | 47.50] loss=3.03 avg=3.10
[30 | 68.84] loss=2.95 avg=3.05
[40 | 90.43] loss=3.39 avg=3.14
[50 | 112.17] loss=2.96 avg=3.10
[60 | 134.09] loss=3.07 avg=3.10
[70 | 156.16] loss=2.63 avg=3.03
[80 | 178.38] loss=3.02 avg=3.03
[90 | 200.75] loss=2.85 avg=3.01
[100 | 223.20] loss=2.69 avg=2.97
[110 | 245.74] loss=2.78 avg=2.95
[120 | 268.32] loss=2.81 avg=2.94
[130 | 291.06] loss=2.77 avg=2.93
[140 | 313.81] loss=2.73 avg=2.91
[150 | 336.63] loss=2.70 avg=2.90
[160 | 359.50] loss=2.55 avg=2.87
[170 | 382.35] loss=2.50 avg=2.85
[180 | 405.17] loss=2.34 avg=2.82
[190 | 427.98] loss=2.51 avg=2.80
[200 | 450.76] loss=2.55 avg=2.79
======== SAMPLE 1 ========
 higher (i.e._), a few are very light, nice to hold/tight.  Some are a little loose or rubbery, but not a big deal.  I really like the texture of this case, especially for those of us who love the feel of a protective case.

I don't want to purchase one for the wallet

Instructions for updating:
Use standard file APIs to delete files with this prefix.


In [18]:
# This command is to compress the trained model in a zip file
import os
from google.colab import files
os.system( "zip -r {} {}".format( 'checkpoint.zip' , 'checkpoint' ) )


0

In [19]:
!ls -lh checkpoint.zip

-rw-r--r-- 1 root root 442M Nov 21 10:11 checkpoint.zip


In [20]:
# This comand is to download the trained model so we don't need to retrain each time we want to use it
files.download('checkpoint.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## 4.1. Load a trained model

In [31]:
#sess = gpt2.start_tf_sess()

In [17]:
#This command is to upload a trained model
gpt2.load_gpt2(sess)

Loading checkpoint checkpoint/run1/model-1000


# 5.Generate text from the pre-trained model

In [18]:
#This is to measure the average lenght of the reviews, so we create Fake reviews with similar lenght
df['Text_length_words'] = df.reviewText.str.split().str.len()   #word lenght
len_media=df['Text_length_words'].mean()
len_max= df['Text_length_words'].max()
len_min= df['Text_length_words'].min()
len_q3=df['Text_length_words'].quantile(0.75)

print('Average lenght: ', len_media)
print( "Max lenght", len_max)
print( "Min lenght", len_min)
print( "Third quartile", len_q3)

Average lenght:  157.222
Max lenght 3412
Min lenght 18
Third quartile 192.25


In [19]:
run_name='run1'
nsamples=60
temperature= 0.7
dataframe_title='CellPhonesAccessories'

In [20]:
#gen_file = 'gpt2_gentext_positive{:%Y%m%d_%H%M%S}.txt'.format(datetime.utcnow())
gen_file= dataframe_title + '_deceptive_GPT2.txt'

gpt2.generate_to_file(sess,
                      run_name=run_name,
                      destination_path=gen_file,
                      length=200,
                      temperature=temperature,
                      nsamples= nsamples,
                      batch_size=20,
                      prefix='<|start|>',   #Add this for us to facilitate finding the beginning of a new review.
                      truncate='<|endoftext|>',
                      include_prefix=False
                      )


In [ ]:
# may have to run twice to get file to download
files.download(gen_file)

# 6.Create the new GPT2 dataset

In [21]:
#Deceptive dataset (new negative and positive gpt2 reviews)

#Adding first row to the dataset
def FirstRow(file, newtext):
    with open(file, 'r+') as f:
        content = f.read()
        f.seek(0, 0)
        f.write(newtext.rstrip('\r\n') + '\n' + content)

In [22]:
#Generated files with GPT2-simple

file=gen_file
FirstRow(file,'====================')

df_fake= pd.read_csv(file, sep='====================',index_col=None, names=['reviewText','deceptive'],header=0)   #\n

#Delete empty rows
df_fake.dropna(subset=['reviewText'], inplace=True)

#Delete rows with character ('"')
indexNames = df_fake[ df_fake['reviewText'] == '"' ].index
df_fake.drop(indexNames , inplace=True)

df_fake['fake']= '1'
df_fake.reset_index(inplace=True)
df_fake=df_fake[['reviewText','fake']]
df_fake.head()

#Delet character ('""') at the beginning of the text
df_fake['reviewText'] = df_fake['reviewText'].map(lambda x: x.lstrip('"'))


/usr/local/lib/python3.7/dist-packages/pandas/util/_decorators.py:311: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  return func(*args, **kwargs)


In [23]:
df_fake

,reviewText,fake
0,"<|start|>,""This headset works great. I used it...",1
1,The bad: The headset does not work properly if...,1
2,The good: The headset works well. It is afford...,1
3,",""So I am a huge fan of hard cases. Every dece...",1
4,This case is a close second with the Galaxy S3...,1
...,...,...
182,"The new design is sleek and new, but the iPhon...",1
183,The,1
184,",""This case is well constructed and fits snugl...",1
185,I'm very pleased with the construction of this...,1


In [24]:
df_truthful

,reviewText,fake
0,I originally ordered screen protectors in Dece...,0
1,I used this item for little over a month and t...,0
2,I wanted something to protect my phone from bo...,0
3,I had a local stereo shop install this item.\n...,0
4,"I've went through a few cases from Case Crown,...",0
...,...,...
495,"So, as many Life Proof case owners, I was sear...",0
496,Pretty decent clear case for iPhone 4/4s. The ...,0
497,I just received this today and wanted to give ...,0
498,I purchased this phone for a couple of months ...,0


In [25]:
#FINALLY WE CREATE THE FINAL DATASET MERGING THE FAKE REVIEWS AND THE REAL ONES.

finaldf = pd.concat([df_truthful,df_fake])
finaldf

,reviewText,fake
0,I originally ordered screen protectors in Dece...,0
1,I used this item for little over a month and t...,0
2,I wanted something to protect my phone from bo...,0
3,I had a local stereo shop install this item.\n...,0
4,"I've went through a few cases from Case Crown,...",0
...,...,...
182,"The new design is sleek and new, but the iPhon...",1
183,The,1
184,",""This case is well constructed and fits snugl...",1
185,I'm very pleased with the construction of this...,1


In [26]:
#Download the dataset

dataset_name= dataframe_title +'deceptivetruthful_dataset.txt'

finaldf.to_csv(dataset_name,index=False)
files.download(dataset_name)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>